In [25]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
import numpy as np

from openai import OpenAI
# from top_secret import my_sk
client = OpenAI(api_key=)

In [26]:
df=pd.read_csv("survey.csv")

In [27]:
df

,job_title,join_reason
0,Founder,"How to fine tune LLM effectively, and what LLM..."
1,Financial Management & Compliance Advisor,Gain knowledge and skills to be able to develo...
2,Founder in the making,Build an AI product
3,Staff Scientist,Better understanding and skill set in AI to ma...
4,Data scientist,Getting skills (for my resume)
...,...,...
78,Technical Project Manager,"AI basics, introduction and how to use it my w..."
79,SAP Technical Consultant,Skill-up on 'Python' and AI.
80,Senior software developer,Developer-level understanding of Gen-AI and re...
81,IT Manager,practical knowledge


In [28]:
df.shape

(83, 2)

In [29]:
df= df.dropna()

In [30]:
df.shape

(66, 2)

In [33]:
job_embeddings_dim = 5
reason_embedding_dim = 24

In [34]:
job_response = client.embeddings.create(
    input=df['job_title'],
    model="text-embedding-3-small",
    dimensions = job_embeddings_dim
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# compute embeddings for joined reasons
reason_response = client.embeddings.create(
    input=df['join_reason'],
    model="text-embedding-3-small",
    dimensions = reason_embedding_dim
)

In [ ]:
# create list with all embeddings
embedding_list = [job_response.data[i].embedding + reason_response.data[i].embedding for i in range(len(df))]

In [ ]:
# create df_embeddings
col_names = [f"job_embedding-{i+1}" for i in range(len(job_response.data[0].embedding))] + [f"reason_embedding-{i+1}" for i in range(len(reason_response.data[0].embedding))]
df_embeddings = pd.DataFrame(embedding_list, columns=col_names)

In [ ]:
df_embeddings

In [ ]:
num_segments = 5
clustering = KMeans(n_clusters=num_segments, random_state=0).fit(df_embeddings)

# add segments to survey data
df["segment"] = clustering.labels_+1

In [ ]:
# use PCA to reduce dimensionality for visualization
X = PCA(n_components=2).fit_transform(df_embeddings)

# plot components for each segment
for label in range(num_segments):
    indexes = np.where(clustering.labels_ == label)[0]
    plt.scatter(X[indexes,0], X[indexes,1], label=f"Segment {label+1}")

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.show()

In [ ]:
def dataframe_to_markdown(df):
    """
    Converts a Pandas DataFrame into a Markdown table.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to be converted into a Markdown table.

    Returns
    -------
    str
        A string representing the DataFrame in Markdown table format.
        The table includes a header row with column names and rows
        formatted with proper Markdown table syntax."""
    
    # Create the header row
    markdown = f"| {' | '.join(df.columns)} |\n"
    markdown += f"| {' | '.join(['-' * len(col) for col in df.columns])} |\n"
    
    # Add each row
    for _, row in df.iterrows():
        markdown += f"| {' | '.join(str(cell) for cell in row)} |\n"
    
    return markdown

In [ ]:
prompt_template = lambda markdown_table: f"""You are a business strategist specializing in customer segmentation and profiling. \
Below is a table of survey responses from customers, including their job titles and reasons for joining an AI bootcamp. \
Your task is to analyze the responses and generate a **single predominant customer profile** that represents the most common \
characteristics and motivations across the group.

Please include:
1. **Job Title**: A single representative job title summarizing those listed in the table
2. **Desired Outcomes**: A single representative desired outcome synthesizing those listed in the table
3. **Key Challenges**: A single representative key challenge synthesizing those listed in the table

### Survey Responses

{markdown_table}                                                                                                                                       |

### Instructions:
- Keep each section concise (1 sentence or bullet points).
- Use simple language and avoid unnecessary details.
- Total response should be under **100 words**.
- Only include the three items list above (i.e. Job Title, Desired Outcomes, and Key Challenges)

Begin your analysis and generate the customer profile below:
"""

In [ ]:
def generate_avatar(my_sk, df_segment):
    """
    Generates a customer avatar by using a GPT model to process a Pandas DataFrame segment.

    Parameters
    ----------
    my_sk : str
        The API key or secret used for authenticating the GPT API client.
    df_segment : pandas.DataFrame
        A segment of a Pandas DataFrame containing relevant customer data 
        that will be used to construct the prompt.

    Returns
    -------
    str
        A string representing the generated customer avatar."""
    
    # construct prompt
    markdown_table = dataframe_to_markdown(df_segment)
    prompt = prompt_template(markdown_table)
    
    # make api call
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ], 
        temperature = 0.5
    )
    
    # extract response
    customer_avatar = response.choices[0].message.content
    return customer_avatar

In [ ]:
for i in range(num_segments):
    survey_data = df[clustering.labels_==i][['job_title', 'join_reason']]
    print("Segment", i+1, "| Size:", len(survey_data))
    print(generate_avatar(my_sk, survey_data))
    print("-" * 100)

In [ ]:
df[df["segment"]==5]